In [1]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import featuretools as ft
import lightgbm as lgb
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from random import sample 
import pickle
import eli5
from eli5.sklearn import PermutationImportance
from lightgbm import LGBMClassifier

%matplotlib inline
RSEED = 50

# Load Original Features

In [2]:
feat_num = 476
#df_total = pd.read_csv('./data/features%s.csv'%(feat_num))
with open('./data/features%s.pickle'%(feat_num), 'rb') as handle:
    df_total = pickle.load(handle)

In [3]:
df_train = df_total[df_total['isFraud'].notnull()]
df_train.shape

(590540, 476)

# Sample Train Data

In [5]:
df_train_sample=df_train.sample(n=200000,random_state=RSEED)
df_train_sample.shape

(200000, 476)

# Prepare Data

In [6]:
labels_train = df_train_sample['isFraud']
features_train = df_train_sample.drop(columns = ['isFraud', 'TransactionID'])
features_train.shape

(200000, 474)

In [7]:
categorical_raw = ['ProductCD', 'card2', 'card3', 'card4', 'card5','card6',
              'addr1','addr2','P_email','R_email','M1','M2','M3',
              'M4','M5','M6','M7','M8','M9','DeviceType','DeviceInfo','dow','hour',
              'Device_name','Device_version','screen_width','screen_height',
               'P_email_suffix','R_email_suffix','id_30_OS','id_30_version',
               'is_card_freq_Device','is_wide','is_long','is_zero','is_win8_vista',
               'is_windows_otheros','is_card_freq_pdc','is_card_freq_addr1'] 

ids = [ 'id_%s'%(i) for i in range(12,39)]
categorical_raw = categorical_raw + ids

In [50]:
#categorical = list(set(categorical_raw).intersection(features_train.columns))
#features_train[categorical].nunique().sort_values(ascending=False)

# Select Features

In [8]:
params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47,
          #'is_unbalance':True
          #'scale_pos_weight':9
         }

In [14]:
def train_selector(params,train_num,features_train,labels_train,categorical,verbose_eval=500):
    train_set = lgb.Dataset(features_train.iloc[0:train_num,:], label=labels_train.values[0:train_num],
                       categorical_feature=categorical)
    valid_set = lgb.Dataset(features_train.iloc[train_num:,:], label=labels_train.values[train_num:],
                       categorical_feature=categorical)
    valid_results = {}
    model = lgb.train(params,train_set,num_boost_round = 10000, 
                   valid_sets = [train_set, valid_set],
                    verbose_eval= verbose_eval,
                    early_stopping_rounds = 500,
                    evals_result=valid_results)
    return model,valid_results

In [10]:
def select_by_importance(model,features_train,importance=0,num_keep=None):
    fi = pd.DataFrame({'feature': features_train.columns, 
                   'importance':model.feature_importance()})
    fi = fi.sort_values('importance', ascending = False)
    if num_keep != None:
        to_drop = fi.iloc[num_keep:,:].feature
    else:
        to_drop = fi[fi.importance <= importance].feature
    return to_drop
    

In [11]:
def fold_train_selector(Nfold,features_train,labels_train,categorical):
    splits = Nfold
    ave_auc = 0
    valid_results = {}
    
    folds = KFold(n_splits = splits,random_state=RSEED)
    
    for fold_num, (trn_idx, val_idx) in enumerate(folds.split(features_train.values, 
                                                          labels_train.values)):
        print("Fold {}".format(fold_num))
        train_df, y_train_df = features_train.iloc[trn_idx], labels_train.iloc[trn_idx]
        valid_df, y_valid_df = features_train.iloc[val_idx], labels_train.iloc[val_idx]
    
        trn_data = lgb.Dataset(train_df, label=y_train_df,categorical_feature=categorical)
        val_data = lgb.Dataset(valid_df, label=y_valid_df,categorical_feature=categorical)
    
        
        clf = lgb.train(params,
                        trn_data,
                        10000,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=500,
                        early_stopping_rounds=500,
                       evals_result=valid_results)
        

        pred = clf.predict(valid_df)
        auc_score = roc_auc_score(y_valid_df, pred)
        ave_auc += auc_score / splits
        print( "  auc = ", auc_score )
    return ave_auc

In [12]:
def fold_select_feature(Nfold,features_train,labels_train,categorical,importance=0):
    splits = Nfold
    ave_auc = 0
    valid_results = {}
    to_drop=[]
    
    folds = KFold(n_splits = splits,random_state=RSEED)
    
    for fold_num, (trn_idx, val_idx) in enumerate(folds.split(features_train.values, 
                                                          labels_train.values)):
        print("Fold {}".format(fold_num))
        train_df, y_train_df = features_train.iloc[trn_idx], labels_train.iloc[trn_idx]
        valid_df, y_valid_df = features_train.iloc[val_idx], labels_train.iloc[val_idx]
    
        trn_data = lgb.Dataset(train_df, label=y_train_df,categorical_feature=categorical)
        val_data = lgb.Dataset(valid_df, label=y_valid_df,categorical_feature=categorical)
    
        
        clf = lgb.train(params,
                        trn_data,
                        10000,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=500,
                        early_stopping_rounds=500,
                       evals_result=valid_results)
        
        drop_this_round = list(select_by_importance(clf,train_df,importance=importance))
        print(drop_this_round)
        
        if fold_num == 0:
            to_drop = drop_this_round
        else:
            to_drop = list(set(to_drop).intersection(drop_this_round))
        print(to_drop)
        
        pred = clf.predict(valid_df)
        auc_score = roc_auc_score(y_valid_df, pred)
        ave_auc += auc_score / splits
        print( "  auc = ", auc_score )
    
    return ave_auc,to_drop
    
    

In [16]:
def permutation_importance(model,features_valid,labels_valid):
    """calculate permutation importance of features
    Args:
        model: the trained model.
        features_valid: dataframe. The validation set of features.
        labels_valid: labels of validation set.
    Returns:
        df_fimportance: dataframe. The importances of features.
    """
    base_score = roc_auc_score(labels_valid, model.predict(features_valid))
    list_fimportance = []
    for col in features_valid.columns:
        #print(col)
        save = features_valid[col].copy()
        features_valid[col] = np.random.permutation(features_valid[col])
        col_score = roc_auc_score(labels_valid, model.predict(features_valid))
        features_valid[col] = save
        list_fimportance.append([col,base_score - col_score])
    return pd.DataFrame(list_fimportance,columns = ['feature','importance']) 
    

## Train with all feature set

In [15]:
train_num = 160000
categorical = list(set(categorical_raw).intersection(features_train.columns))
model,valid_results = train_selector(params,train_num,features_train,labels_train,
                                     categorical,verbose_eval=500)

/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.996283	valid_1's auc: 0.940631
[1000]	training's auc: 0.999922	valid_1's auc: 0.950368
[1500]	training's auc: 1	valid_1's auc: 0.952541
[2000]	training's auc: 1	valid_1's auc: 0.953066
Early stopping, best iteration is:
[1714]	training's auc: 1	valid_1's auc: 0.95285


## Permutation Importance

In [17]:
fi_importance = permutation_importance(model,features_train.iloc[160000:],labels_train.iloc[160000:])

In [19]:
fi_importance.sort_values('importance',ascending=False)

,feature,importance
4,card2,0.011661
9,addr1,0.008445
25,C13,0.006763
0,TransactionDT,0.003501
26,C14,0.003312
13,C1,0.003181
427,P_email,0.002904
7,card5,0.002088
3,card1,0.001852
437,pdc_amt_ratio,0.001575


In [20]:
categorical = list(set(categorical_raw).intersection(features_train.columns))
ave_auc,to_drop = fold_select_feature(3,features_train,labels_train,categorical,importance=0)

Fold 0


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.996758	valid_1's auc: 0.936508
[1000]	training's auc: 0.99995	valid_1's auc: 0.945776
[1500]	training's auc: 1	valid_1's auc: 0.947953
[2000]	training's auc: 1	valid_1's auc: 0.948659
Early stopping, best iteration is:
[1589]	training's auc: 1	valid_1's auc: 0.948102
['addr2', 'V119', 'V28', 'is_windows_otheros', 'V240', 'V241', 'V305', 'V122', 'V120', 'V118', 'V117', 'V107', 'id_14', 'V89', 'id_21', 'id_24', 'id_25', 'V88', 'is_win8_vista', 'is_zero']
['addr2', 'V119', 'V28', 'is_windows_otheros', 'V240', 'V241', 'V305', 'V122', 'V120', 'V118', 'V117', 'V107', 'id_14', 'V89', 'id_21', 'id_24', 'id_25', 'V88', 'is_win8_vista', 'is_zero']
  auc =  0.9481015677503738
Fold 1


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.997222	valid_1's auc: 0.932213
[1000]	training's auc: 0.999964	valid_1's auc: 0.941477
[1500]	training's auc: 1	valid_1's auc: 0.943575
[2000]	training's auc: 1	valid_1's auc: 0.943944
Early stopping, best iteration is:
[1723]	training's auc: 1	valid_1's auc: 0.943834
['V240', 'V122', 'V121', 'V120', 'V241', 'V119', 'V118', 'V117', 'V65', 'V305', 'V114', 'is_win8_vista', 'is_windows_otheros', 'id_26', 'id_25', 'id_24', 'id_21', 'id_14', 'V89', 'V107', 'V108', 'V28', 'V27', 'V111', 'V68', 'V113', 'is_zero']
['id_24', 'V120', 'V89', 'V117', 'id_14', 'id_21', 'V305', 'id_25', 'is_win8_vista', 'is_windows_otheros', 'V28', 'is_zero', 'V119', 'V107', 'V240', 'V118', 'V241', 'V122']
  auc =  0.9438337936852855
Fold 2


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.996658	valid_1's auc: 0.93433
[1000]	training's auc: 0.999946	valid_1's auc: 0.942958
[1500]	training's auc: 1	valid_1's auc: 0.945112
[2000]	training's auc: 1	valid_1's auc: 0.945699
Early stopping, best iteration is:
[1696]	training's auc: 1	valid_1's auc: 0.945423
['V241', 'V240', 'id_22', 'is_windows_otheros', 'is_win8_vista', 'V89', 'V269', 'id_26', 'id_25', 'id_24', 'id_21', 'id_14', 'V107', 'V117', 'V118', 'V119', 'V120', 'V305', 'is_zero']
['id_24', 'V120', 'V117', 'id_14', 'id_21', 'id_25', 'V305', 'is_win8_vista', 'is_windows_otheros', 'is_zero', 'V118', 'V119', 'V107', 'V240', 'V89', 'V241']
  auc =  0.9454225532357191


In [21]:
ave_auc

0.9457859715571262

## Feature selection by Importance

In [22]:
#to_drop = list(select_by_importance(model,features_train,importance=0))
to_drop = ['V91']

In [23]:
features_train_temp = features_train.drop(to_drop,axis=1)
categorical_temp = list(set(categorical_raw).intersection(features_train_temp.columns))
ave_auc = fold_train_selector(3,features_train_temp,labels_train,categorical_temp)

Fold 0


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.996728	valid_1's auc: 0.936943
[1000]	training's auc: 0.99995	valid_1's auc: 0.946019
[1500]	training's auc: 1	valid_1's auc: 0.947929
[2000]	training's auc: 1	valid_1's auc: 0.948716
Early stopping, best iteration is:
[1594]	training's auc: 1	valid_1's auc: 0.948128
  auc =  0.9481276527650302
Fold 1


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.997175	valid_1's auc: 0.932507
[1000]	training's auc: 0.999964	valid_1's auc: 0.941967
[1500]	training's auc: 1	valid_1's auc: 0.94389
[2000]	training's auc: 1	valid_1's auc: 0.944174
Early stopping, best iteration is:
[1699]	training's auc: 1	valid_1's auc: 0.94413
  auc =  0.9441302938301871
Fold 2


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.996641	valid_1's auc: 0.934443
[1000]	training's auc: 0.999948	valid_1's auc: 0.943246
[1500]	training's auc: 1	valid_1's auc: 0.945512
[2000]	training's auc: 1	valid_1's auc: 0.946168
Early stopping, best iteration is:
[1676]	training's auc: 1	valid_1's auc: 0.945838
  auc =  0.9458383939643953


In [24]:
ave_auc

0.9460321135198708

In [105]:
with open('./data/feat476_rm_importance0.pickle', 'wb') as handle:
    pickle.dump(to_drop, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Recursive Eliminate Features

In [ ]:
best = 0.9531365218167643
rm_col = ''
for col in categorical:
    print(col)
    features_train_temp = features_train.drop(col,axis=1)
    categorical_temp = list(set(categorical_raw).intersection(features_train_temp.columns))
    model_temp,valid_results = train_selector(params,train_num,features_train_temp,labels_train,categorical_temp,verbose_eval=2000)
    pfm = max(valid_results['valid_1']['auc'])
    if pfm > best:
        print('remove %s improve auc from %s to %s this feature is endanger'%(col,best,pfm))
        best = pfm
        rm_col = col
        

Device_name


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[2000]	training's auc: 1	valid_1's auc: 0.953302
Early stopping, best iteration is:
[1729]	training's auc: 1	valid_1's auc: 0.953183
remove Device_name improve auc from 0.9531365218167643 to 0.9534075096476156 this feature is endanger
id_26


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[2000]	training's auc: 1	valid_1's auc: 0.952548
Early stopping, best iteration is:
[1722]	training's auc: 1	valid_1's auc: 0.952363
P_email


...

Training until validation scores don't improve for 500 rounds.
[2000]	training's auc: 1	valid_1's auc: 0.952821
Early stopping, best iteration is:
[1826]	training's auc: 1	valid_1's auc: 0.952705
M8


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[2000]	training's auc: 1	valid_1's auc: 0.953178
Early stopping, best iteration is:
[1741]	training's auc: 1	valid_1's auc: 0.953064
is_windows_otheros


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.


In [64]:
rm_col

'ProductCD'